## Importing Libraries

In [1]:
from fastai.text.all import *

## Load Data

In [2]:
path = Path("../data/identify_the_sentiments/")

In [3]:
train_df = pd.read_csv(path/'train_2kmZucJ.csv')
train_df = train_df.rename(columns={'tweet':'text'})
test_df = pd.read_csv(path/'test_oJQbWVk.csv')
test_df = test_df.rename(columns={'tweet':'text'})


In [4]:
train_df.head()

,id,label,text
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [5]:
test_df.head()

,id,text
0,7921,I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks
1,7922,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://instagr.am/p/UUIS0bIBZo/
2,7923,"I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the screen?\n"
3,7924,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and from Vet Camp #hatinglife #sobbing"
4,7925,Been fighting iTunes all night! I only want the music I $&@*# paid for


## Training a Language Model

In [6]:
## Combining all the textual data
df_all = pd.concat([train_df.drop(['label'], axis=1),test_df])

In [7]:
## Creating a Text Data loader for language model
dls_lm = TextDataLoaders.from_df(df_all, text_col=1, is_lm=True)
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos a xxrep 3 h all i want to do is save all my photos on the cloud somewhere from my ipad but it 's xxunk impossible xxbos xxmaj xxunk # xxmaj siri , you 're so sweet , now i love ya even more … # iphone http : / / instagram.com / p / xxunk / xxbos … awesome . xxmaj the new xxmaj mac update for iphoto now means","a xxrep 3 h all i want to do is save all my photos on the cloud somewhere from my ipad but it 's xxunk impossible xxbos xxmaj xxunk # xxmaj siri , you 're so sweet , now i love ya even more … # iphone http : / / instagram.com / p / xxunk / xxbos … awesome . xxmaj the new xxmaj mac update for iphoto now means i"
1,xxunk xxunk % . # xxunk xxbos xxmaj going to see if i can xxunk my $ & @ * # iphone for another phone ! xxmaj all ios7 's fault ! # apple # xxunk xxbos xxmaj new macbook xxmaj pro xxmaj retina ! ! # apple # macbookpro # retina # life # business # work # spring … https : / / instagram.com / p / xxunk / xxbos,xxunk % . # xxunk xxbos xxmaj going to see if i can xxunk my $ & @ * # iphone for another phone ! xxmaj all ios7 's fault ! # apple # xxunk xxbos xxmaj new macbook xxmaj pro xxmaj retina ! ! # apple # macbookpro # retina # life # business # work # spring … https : / / instagram.com / p / xxunk / xxbos xxmaj
2,"click here https : / / tinyurl.com / xxunk pic.twitter.com / xxunk xxbos 10 minutes ago my battery was at 35 % .. currently ? ? 8 xxunk percent . # iphone xxbos # xxunk finally fixed , so fast , so xxunk ! xxmaj thx # <3 # oneplus xxbos xxmaj who are you waiting for ? # wait # you # cute # afternoon # iphone # xxunk # dog","here https : / / tinyurl.com / xxunk pic.twitter.com / xxunk xxbos 10 minutes ago my battery was at 35 % .. currently ? ? 8 xxunk percent . # iphone xxbos # xxunk finally fixed , so fast , so xxunk ! xxmaj thx # <3 # oneplus xxbos xxmaj who are you waiting for ? # wait # you # cute # afternoon # iphone # xxunk # dog #"
3,"blue # apple http : / / instagram.com / p / xxunk / xxbos xxmaj welcome to my world , iphone 6 =] # iphone6 # iphone # apple # life # xxunk http : / / instagram.com / p / xxunk / xxbos xxmaj luxury xxmaj fundas xxmaj woo … and much more at xxmaj zeta xxmaj supplies . xxmaj click the link - http : / / zetasupplies.co.uk / products","# apple http : / / instagram.com / p / xxunk / xxbos xxmaj welcome to my world , iphone 6 =] # iphone6 # iphone # apple # life # xxunk http : / / instagram.com / p / xxunk / xxbos xxmaj luxury xxmaj fundas xxmaj woo … and much more at xxmaj zeta xxmaj supplies . xxmaj click the link - http : / / zetasupplies.co.uk / products /"
4,home # iphone # ipad # gadgets # gifts # ios # cars # bike # xxmaj android # xxmaj samsung # galaxy # xxmaj google http : / / crwd.fr / xxunk pic.twitter.com / xxunk xxbos # its # me # xxmaj xxunk # orange # pictureoftheday # samsung # xxunk # galaxy # xxup s2 # blonde xxunk … http : / / instagram.com / p / xxunk / xxbos,# iphone # ipad # gadgets # gifts # ios # cars # bike # xxmaj android # xxmaj samsung # galaxy # xxmaj google http : / / crwd.fr / xxunk pic.twitter.com / xxunk xxbos # its # me # xxmaj xxunk # orange # pictureoftheday # samsung # xxunk # galaxy # xxup s2 # blonde xxunk … http : / / instagram.com / p / xxunk / xxbos xxmaj


In [8]:
## Initiating the language model
learn = language_model_learner(
    dls_lm, 
    AWD_LSTM, 
    metrics=[accuracy, Perplexity()], 
    path=path, 
    wd=0.1).to_fp16()

In [9]:
## Training last layer
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.027797,3.592271,0.397588,36.316452,00:16
1,4.018194,3.191223,0.444724,24.318148,00:15
2,3.559542,3.027364,0.464616,20.642757,00:15
3,3.326017,2.967018,0.471579,19.433884,00:15
4,3.208390,2.957968,0.472239,19.258808,00:15


In [11]:
## Training all layers
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.083944,2.874427,0.483968,17.715277,00:19
1,2.951321,2.760319,0.497860,15.804892,00:19
2,2.825868,2.801350,0.492643,16.466856,00:18
3,2.724147,2.616971,0.521871,13.694175,00:19
4,2.617326,2.579108,0.527438,13.185374,00:18
5,2.531577,2.542779,0.532831,12.714956,00:18
6,2.458514,2.516879,0.537753,12.389866,00:18
7,2.410636,2.506086,0.541144,12.256858,00:19
8,2.368439,2.501094,0.541619,12.195827,00:18
9,2.366574,2.501034,0.541729,12.195101,00:18


In [12]:
## Saving encoder 
learn.save_encoder('finetuned')

## Training a Classifier

In [13]:
## Creating a dataloader 
dls_clas = TextDataLoaders.from_df(
    train_df, 
    valid_pct=0.2,
    seed=42,
    text_col=2,
    label_col=1,
    text_vocab=dls_lm.vocab)

In [14]:
## Checking if everything is working fine
dls_clas.show_batch(max_n=5)

,text,category
0,xxbos xxup xxunk xxmaj bluetooth xxmaj smartwatch xxmaj smart xxmaj watch with xxup sim xxmaj card xxmaj slot and xxunk xxmaj camera for iphone / xxmaj samsung and xxmaj android xxmaj phones https : / / xxunk / products / xxunk - smartwatch - smart - watch - with - sim - card - slot - xxunk - xxunk - camera - for - iphone - samsung - and - android - phones … # xxmaj bluetooth # xxmaj smartwatch # xxmaj smart # xxmaj watch # xxup sim # xxmaj card # xxmaj slot # xxmaj camera # iphone # xxmaj samsung # xxmaj android # xxmaj phones # sports # cute # pretty,0
1,xxbos xxmaj use xxmaj google xxmaj assistant on xxmaj your iphone xxmaj even xxmaj if xxmaj it 's xxmaj not xxmaj on the xxmaj xxunk # siri # apple # ios # iphone # ipad # smartphones # mobile # tech # info # howto # nice # google # xxunk # xxunk # youtube # netflix # nice # productivity # xxunk https : / / techinwire.com / 2018 / 02 / 09 / use - google - assistant - on - your - iphone - even - if - its - not - on - the - xxunk / … pic.twitter.com / xxunk,0
2,"xxbos xxmaj sleep : a xxmaj key xxmaj xxunk to xxmaj happy , xxmaj healthy and xxmaj productive xxmaj lives .. https : / / xxrep 3 w xxunk / monthly - xxunk / health - link - new / xxunk - a - key - xxunk - to - happy - healthy - and - productive - lives … # productive # healthy # health # glad # smile # xxunk # monthly # daily # article # health # healthy # mac # apple # working # xxunk # work # xxunk # thursday # ha xxrep 5 p xxunk / xxunk",0
3,"xxbos xxmaj if xxmaj you xxmaj like xxmaj sports , xxmaj those 3 xxmaj games xxmaj should xxmaj be in xxmaj your xxmaj pocket # game # gaming # xxunk # mobile # smartphone # app # google # apple # ios # android # trend # tech # enjoy # soccer # basketball # xxmaj xxunk # sports # competition # winning # relax # family https : / / techinwire.com / 2018 / 02 / 16 / if - you - like - sports - xxunk - should - be - in - your - pocket / … pic.twitter.com / xxunk",0
4,xxbos xxmaj xxunk xxmaj blue xxmaj xxunk xxmaj bird xxmaj xxunk - xxmaj gardening xxmaj gifts - xxmaj xxunk xxmaj gifts - xxmaj birds - xxmaj apple https : / / xxrep 3 w xxunk / product / xxunk - blue - xxunk - bird - xxunk - gardening - gifts - xxunk - gifts - birds - apple - xxunk - scotland - xxunk - nature - garden / … # xxunk # xxunk # xxunk # xxunk # birds # xxunk # gifts # xxunk # xxunk # xxmaj apple # green # xxup blue # xxmaj xxunk # xxunk,0


In [15]:
## Defining our text classifier
learn = text_classifier_learner(dls_clas , AWD_LSTM, drop_mult=1, metrics=accuracy)
learn.path = path

In [16]:
## Loading Language model encoder weights trained in previous section
learn = learn.load_encoder(file='finetuned')

In [17]:
## Finetuning last layer
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.653715,0.242876,0.898359,00:10


In [18]:
## Finetuning last 2 layers and progressive learning rate
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.531201,0.229384,0.904040,00:12


In [19]:
## Finetuning last 3 layers and progressive learning rate
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.428972,0.229589,0.898990,00:14


In [20]:
## Training entire model
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.410783,0.222436,0.907197,00:16
1,0.392694,0.220471,0.904040,00:16
2,0.383803,0.223444,0.905934,00:16
3,0.389135,0.234163,0.901515,00:16
4,0.370273,0.221122,0.905934,00:16


In [21]:
## Save model weights
learn.save('classifier')

Path('../data/identify_the_sentiments/models/classifier.pth')

## Making predictions

In [22]:
## Getting testing data and doing predictions
test_dl = learn.dls.test_dl(test_df)
preds, _ = learn.get_preds(dl=test_dl)

In [23]:
## Creating submission file
submission = pd.DataFrame({'id':test_df.iloc[test_dl.get_idxs(),0].values})
submission['label'] = preds.argmax(axis=1)

Note above we have to get indexes for our DL for test set. This is because the data loader shuffles the dataset in FastAI and we need to find the predictions we get our for which row in training set.

In [24]:
## Exporting predictions
submission.to_csv("./submission/submission_ulmfit.csv", index=False)